In [11]:
import pathlib
import pandas as pd

project_path = pathlib.Path("/mnt/disk01/insight-volume/MontoPirelli")

# directions = ["000", "045", "090", "135", "180", "225", "270", "315"]
directions = ["045", "090", "135", "180", "225", "270", "315"]

root_cp_path = project_path / "G100" / "cases" / "000" / "generated" / "cp" / "default_cp" / "f4f9aae9cf8a9da32466908c93ddd63c5a0c26b74ab4ba9ec377e6377c17bed2" / "stats_df.h5"

max_cp_path = project_path / "Env_max" / "cases" / "cp_max.h5"
min_cp_path = project_path / "Env_min" / "cases" / "cp_min.h5"

root_cp_stats = pd.read_hdf(root_cp_path)
root_cp_stats.drop(columns=["cp_mean", "cp_std", "cp_mean_qs"], inplace=True)

for direct in directions:
    cp_path = project_path / "G100" / "cases" / direct / "generated" / "cp" / "default_cp" / "f4f9aae9cf8a9da32466908c93ddd63c5a0c26b74ab4ba9ec377e6377c17bed2" / "stats_df.h5"
    cp_stats = pd.read_hdf(cp_path)
    cp_stats.drop(columns=["cp_mean", "cp_std", "cp_mean_qs"], inplace=True)
    cp_stats.rename(columns={"cp_xtr_min" : f"cp_xtr_min_{direct}", "cp_xtr_max" : f"cp_xtr_max_{direct}"}, inplace=True)
    root_cp_stats = pd.merge(root_cp_stats, cp_stats, on=["point_idx"])
    
root_cp_stats['max_cp'] = root_cp_stats[[col for col in root_cp_stats.columns if "cp_xtr_max" in col]].max(axis=1)
root_cp_stats['min_cp'] = root_cp_stats[[col for col in root_cp_stats.columns if "cp_xtr_min" in col]].min(axis=1)
root_cp_stats.drop(columns=[col for col in root_cp_stats.columns if col not in ["point_idx", "max_cp", "min_cp"]], inplace=True)
root_cp_stats.rename(columns={"max_cp": "cp_xtr_max", "min_cp": "cp_xtr_min"}, inplace=True)
root_cp_stats.drop(columns=["cp_xtr_min"]).to_hdf(max_cp_path, key="data")
root_cp_stats.drop(columns=["cp_xtr_max"]).to_hdf(min_cp_path, key="data")
root_cp_stats

,point_idx,cp_xtr_max,cp_xtr_min
0,0,0.944356,-1.722007
1,1,0.739893,-0.733508
2,2,1.022465,-1.307036
3,3,0.594335,-2.161281
4,4,1.264401,-1.133866
...,...,...,...
103473,103473,0.613239,-0.783370
103474,103474,0.569613,-0.747599
103475,103475,0.559199,-0.662064
103476,103476,0.640086,-0.821070


In [17]:
from cfdmod.use_cases.pressure.cp_config import CpCaseConfig
import pathlib

cfg_path = pathlib.Path("/mnt/disk01/insight-volume/MontoPirelli/Env_max/configs/cp_params.yaml")
post_proc_cfg = CpCaseConfig.from_file(cfg_path)

for cfg_lbl, cfg in post_proc_cfg.pressure_coefficient.items():
    print(cfg_lbl, cfg.sha256())

default_cp 2f1b96a9b483f9ecf91abf484e3c074cd1e803ff2109a610a0824dd9696adea8


In [3]:
from cfdmod.use_cases.pressure.path_manager import CpPathManager
import pathlib
from cfdmod.logger import logger
from cfdmod.use_cases.pressure.cp_data import process_cp
from lnas import LnasFormat
import pandas as pd
from cfdmod.use_cases.pressure.cp_config import CpCaseConfig
from cfdmod.api.vtk.write_vtk import create_polydata_for_cell_data, write_polydata

project_path = pathlib.Path("/mnt/disk01/insight-volume/MontoPirelli")

case_dict = {
    "Env_max" : ["max"],
    "Env_min" : ["min"],
}

for block_lbl, case_list in case_dict.items():
    for case_lbl in case_list:
        case_hash = "2f1b96a9b483f9ecf91abf484e3c074cd1e803ff2109a610a0824dd9696adea8" if case_lbl == "max" else "2053c8230ed4e7e68bd6dc6e9d62bf37cdc50a33500fc49d91bce536802e1bc4"
        data_path = project_path / block_lbl / "cases" / case_lbl / "generated" / "cp" / "default_cp" / case_hash / f"cp_{case_lbl}.h5"
        vtp_path = project_path / block_lbl / "cases" / case_lbl / "generated" / "cp" / "default_cp" / case_hash / "stats.vtp"
        stats = pd.read_hdf(data_path)
        
        mesh_path = project_path / block_lbl / "mesh" / "G100.merged.lnas"
        mesh = LnasFormat.from_file(mesh_path)

        polydata = create_polydata_for_cell_data(data=stats, mesh=mesh.geometry)
        write_polydata(vtp_path, polydata)